In [ ]:
import sys
from torch.utils.data import random_split, DataLoader 
sys.path.append("../")

In [ ]:
from datasets import datasets
import constants
import pickle
import os
import collections
import pandas
from PIL import Image
import matplotlib.pyplot as plt
import numpy
import cv2
from feature_extraction import feature_extraction
from feature_extraction import texture_analysis

Loading images from folders

In [ ]:
image_paths = collections.defaultdict(list)

for path in os.listdir("../data/raw_data"):

    class_name = path.lower()
    full_path = os.path.join("../data/raw_data", class_name)

    for fil in os.listdir(full_path):
        image_paths[class_name].append(os.path.join(full_path, fil))

Representing number of images for each individual class

In [ ]:
for class_name, paths in image_paths.items():
    print(class_name, 'number of images: ', len(paths))

Creating dataframe of images

In [ ]:
image_dataset = pandas.DataFrame(
    image_paths,
    columns=['class', 'path']
)

# exploding dataset for extracting each individual object
image_dataset['path'] = image_dataset['path'].explode()

# Inserting actual image objects inside the dataframe
image_dataset['image'] = image_dataset['path'].apply(
    lambda item: Image.open(item)
)

Image Visualization

In [ ]:
def visualize_k_random_images(images, k):
    """
    Function visualizes k images, extracted from given source
    
    Args:
        images - typing.List[str] - array of images
        k - number of images
    """
    _, ax = plt.subplots(2, images // k)
    k_random_images = numpy.random.choice(size=2 * images // k, a=images)
    for col in range(ax.shape[0]):
        for row in range(ax.shape[0]):
            ax[col, row].imshow(k_random_images.pop())
        
visualize_k_random_images(image_dataset['Image'])

Removing Noise from images using smoothing filters

In [ ]:
def apply_median_filter(image: numpy.ndarray, kernel_size: int):
    """
    Function applies standard non-linear median filter 
    to image for removing salt-and-papper noise from image
    """
    if len(image) == 0: return 
    filtered_img = cv2.medianBlur(
        src=image,
        ksize=kernel_size
    )
    return filtered_img

def apply_bilateral_filtering(
    image: numpy.ndarray, 
    kernel_size: int,
    sigma_space: int, 
    sigma_color: int
):
    """
    Function applies bilateral filter to given image
    """
    smoothed_img = cv2.bilateralFilter(
        src=image,
        sigmaColor=sigma_color,
        sigmaSpace=sigma_space,
        d=kernel_size,
    )
    return smoothed_img

def sharpen_image(image, blurred_img, sharp_factor: int = 1.5):
    """
    Function defines edges of the image
    using given versions of original and blurred img
    """
    new_img = image - blurred_img 
    img = image + (sharp_factor * new_img)
    return img

Splitting images into noisy and clear ones

In [ ]:
def is_noisy(self, image, threshold: int = 10):
    scaled_img = cv2.cvtColor(image, cv2.IMREAD_GRAYSCALE)
    variance = cv2.Laplacian(src=scaled_img, ).var()
    return variance < threshold

In [ ]:
noisy_images = image_dataset[is_noisy(image_dataset['image'])]['image']

Visualizing noisy images

In [ ]:
copied_images = noisy_images.copy()
plot, ax = plt.subplots()

for col in enumerate(noisy_images.shape[0]):
    for row in range(noisy_images.shape[0]):
        ax[col, row].imshow(noisy_images)
plt.show()

Smoothing noisy images

In [ ]:
for idx, image in enumerate(noisy_images.flatten()):
    # Applying filter to noisy image
    blurred_img = apply_bilateral_filtering(
        image=image,
        sigma_color=30,
        sigma_space=30,
        kernel_size=5
    )
    # sharpening after smoothing
    sharpen_img = sharpen_image(
        image=image, 
        blurred_img=blurred_img,
        sharp_factor=2
    )
    image_dataset.iloc[idx, 'image'] = sharpen_image

Texture Analysis

In [ ]:
image_dataset['texture_features'] = image_dataset['image'].apply(
lambda image: texture_analysis.get_texture_features(image=image))

Splitting data into training and testing sets

In [ ]:
train_size = int(image_dataset['image'].shape[0] * 0.7)
test_size = int(image_dataset['image'].shape[0] * 0.3)

train_data, test_data = random_split(
    image_dataset, 
    [train_size, test_size]
)

Forming datasets

In [ ]:
train_dataset = datasets.FaceRecognitionDataset(
    images=train_data['path'],
    labels=train_data['class']
)

test_dataset = datasets.FaceRecognitionDataset(
    images=test_data['path'],
    labels=test_data['class']
)

Image Processing Evaluation

In [ ]:
def ssim_score(orig_img, blur_img):
    pass

def ms_ssim_score(orig_img, blur_img):
    pass 

def niqe(trans_image):
    pass

def brisque(trans_image):
    pass

In [ ]:
def evaluate_transformed_images(actual_image: Image.Image, transformed_image: Image.Image):
    """
    Function evaluates quality of transformed image 
    using following set of metrics:
        1. SSIM (Structural Simularity Index)

    Args:
        actual_image (Image.Image) - actual (original) version of the image
        transformed_image: (Image.Image) - image for quality evaludation that has been transformed
    """
    metrics = {}
    metrics['niqe_score'] = niqe()
    metrics['brisque_score'] = brisque()
    return metrics

def evaluate_recovered_images(trans_image):
    """
    Function evaluates quality of the image, that has been 
    recovered from the noise using smoothing filter 
    and other techniques
    """
    metrics = {}
    metrics['ssim'] = ssim_score()
    metrics['ms_ssim'] = ms_ssim_score()
    return metrics

In [ ]:
training_loader = DataLoader(
    dataset=train_dataset, 
    batch_size=constants.BATCH_SIZE,
    shuffle=True
)

testing_loader = DataLoader(
    dataset=test_dataset,
    batch_size=constants.BATCH_SIZE,
    shuffle=True
)

Saving datasets to pickle format

In [ ]:
pickle.dumps(training_loader, "../data/processed_data/training_loader.pkl")
pickle.dumps(testing_loader, "../data/processed_data/testing_loader.pkl")